In [1]:
import pandas as pd
import numpy as np

In [2]:
# 영화에 대한 평가 점수를 dictionary 형태로 입력

ratings={
     'A':{'달콤한인생':5,'범죄도시':3,'도둑들':3,'모아나':4},
     'B':{'달콤한인생':5,'범죄도시':1,'도둑들':4,'모아나':4},
     'C':{'달콤한인생':0,'범죄도시':4,'도둑들':5,'모아나':3},
     'D':{'달콤한인생':2,'범죄도시':1,'도둑들':5,'모아나':4}
}

In [4]:
users = 'A,B,C,D'.split(',')

result = dict()
for user in users:
    for k, v in ratings[user].items():
        if k not in result.keys():
            result[k] = v
        else:
            result[k] += v
result

{'달콤한인생': 12, '범죄도시': 9, '도둑들': 17, '모아나': 15}

In [5]:
for movie, rate in result.items():
    result[movie] = rate / len(users)
result

{'달콤한인생': 3.0, '범죄도시': 2.25, '도둑들': 4.25, '모아나': 3.75}

In [6]:
result = sorted(result.items(), key=lambda x: x[1], reverse=True)
result

[('도둑들', 4.25), ('모아나', 3.75), ('달콤한인생', 3.0), ('범죄도시', 2.25)]

# 협업필터링

In [12]:
# 영화에 대한 평가 점수가 다음과 같다

movies={
     'A':{'달콤한인생':5,'범죄도시':3,'모아나':4},
     'B':{'달콤한인생':5,'범죄도시':1,'도둑들':4,'모아나':4, '서치':3},
     'C':{'달콤한인생':1,'범죄도시':4,'도둑들':5,'모아나':3, '명당':2},
     'D':{'달콤한인생':2,'범죄도시':1}
}

## User-Based_CF

In [7]:
# Step1) User - Item matrix 만들기

In [8]:
users

['A', 'B', 'C', 'D']

In [10]:
user_based_cf = pd.DataFrame()
user_based_cf['users'] = users
user_based_cf

,users
0,A
1,B
2,C
3,D


In [18]:
movie_names = []
for movie_rates in movie.values():
    for movie in movie_rates.keys():
        if movie not in movie_names:
            movie_names.append(movie)
movie_names

['달콤한인생', '범죄도시', '모아나', '도둑들', '서치', '명당']

In [22]:
user_based_cf['users'][0]

'A'

In [31]:
movie_rate_dict = dict()
for movies_rates in movies.values():
    for movie, rate in movies_rates.items():
        if movie not in movie_rate_dict.keys():
            movie_rate_dict[movie] = [rate]
        else:
            movie_rate_dict[movie].append(rate)
movie_rate_dict

{'달콤한인생': [5, 5, 1, 2],
 '범죄도시': [3, 1, 4, 1],
 '모아나': [4, 4, 3],
 '도둑들': [4, 5],
 '서치': [3],
 '명당': [2]}

In [30]:
for movie, rates in movie_rate_dict.items():
    for i in user_based_cf.index:
        if i <= len(rates):
            user_based_cf[movie][i] = rates[i]
        else:
            user_based_cf[movie][i] = np.nan
user_based_cf

C:\Users\rlath\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


KeyError: '모아나'

별로 좋지 않은 코드였다 윽

누가 몇 점을 줬는지 알기 어렵기 때문..!

In [45]:
# 엄청 간단한 방법!
df_movie = pd.DataFrame.from_dict(movies, orient='index')
df_movie

,달콤한인생,범죄도시,모아나,도둑들,서치,명당
A,5,3,4.0,NaN,NaN,NaN
B,5,1,4.0,4.0,3.0,NaN
C,1,4,3.0,5.0,NaN,2.0
D,2,1,NaN,NaN,NaN,NaN


In [46]:
df_movie = df_movie.fillna(0)
df_movie

,달콤한인생,범죄도시,모아나,도둑들,서치,명당
A,5,3,4.0,0.0,0.0,0.0
B,5,1,4.0,4.0,3.0,0.0
C,1,4,3.0,5.0,0.0,2.0
D,2,1,0.0,0.0,0.0,0.0


In [32]:
# Step2) 'D' 유저와 다른 유저들의 유사도 측정

## Item-Based_CF

In [33]:
df_movie

,달콤한인생,범죄도시,모아나,도둑들,서치,명당
A,5,3,4.0,0.0,0.0,0.0
B,5,1,4.0,4.0,3.0,0.0
C,1,4,3.0,5.0,0.0,2.0
D,2,1,0.0,0.0,0.0,0.0


In [37]:
movie_corr = df_movie.corr()
movie_corr

,달콤한인생,범죄도시,모아나,도둑들,서치,명당
달콤한인생,1.000000,-0.350329,0.619268,-0.261291,0.565916,-0.727607
범죄도시,-0.350329,1.000000,0.381529,0.316862,-0.555556,0.777778
모아나,0.619268,0.381529,1.000000,0.418472,0.440225,0.088045
도둑들,-0.261291,0.316862,0.418472,1.000000,0.443607,0.697097
서치,0.565916,-0.555556,0.440225,0.443607,1.000000,-0.333333
명당,-0.727607,0.777778,0.088045,0.697097,-0.333333,1.000000


In [38]:
D_sim = movie_corr[['달콤한인생', '범죄도시']]
D_sim

,달콤한인생,범죄도시
달콤한인생,1.000000,-0.350329
범죄도시,-0.350329,1.000000
모아나,0.619268,0.381529
도둑들,-0.261291,0.316862
서치,0.565916,-0.555556
명당,-0.727607,0.777778


In [40]:
(D_sim['달콤한인생'] * 2 + D_sim['범죄도시'] * 1).sort_values(ascending=False)

달콤한인생    1.649671
모아나      1.620065
서치       0.576277
범죄도시     0.299342
도둑들     -0.205720
명당      -0.677436
dtype: float64